## Implementation of Metropolis Hastings to approximate probability queries from a Bayesian Network.

In [1]:
import random
import copy
import numpy as np
from calculations_helper import perform_gibbs_sampling, perform_likelihood_weighting, find_weight
from bayesian_network_helper import topologically_sort

def estimate_metropolis_hastings(p: float, iterations: int, network: dict, queries: list[int], evidence: dict[int,bool]) -> np.array:
    """Generate an estimate for the probability distribution for a given set of query variables and evidence values

    Args:
        p (float): determines probability of generating the next state via either Gibbs Sampling or Likelihood Weighting
        iterations (int): number of samples to take before we go with the estimate
        network (dict): underlying bayesian network
        queries (list[int]): list of query variables
        evidence (dict[int,bool]): list of evidence variables with their respective values

    Returns:
        np.array: estimated probability distribution for the different combinations the query variables can take on
    """
    # We'll use the alternative method fo metropolis hastings described in the README
    prob_distribution = np.zeros(shape=1 << len(queries))

    # create list of non-evidence variables
    non_evidence_variables = [int(i) for i in network.keys() if int(i) not in evidence.keys()]

    # also create list of topologically sorted variables
    sorted_vars = topologically_sort(network=network)

    prev_weighted_likelihood = 0.0
    for _ in range(iterations):
        current_evidence = copy.deepcopy(evidence)
        if random.random() < p:
            # randomly initialize all the variables
            for v in non_evidence_variables:
                current_evidence[v] = (random.random() < 0.5)
            # Perform Gibbs sampling for our next state and accept it
            posn = perform_gibbs_sampling(non_evidence_variables, current_evidence, network, queries)
            prev_weighted_likelihood = find_weight(current_evidence, evidence, network)
            # update the distribution accordingly
            prob_distribution[posn] += 1.0
        else:
            # Perform likelihood weighting for our next state, but only move to it if the state is more likely than our previous state
            old_evidence = copy.deepcopy(evidence)
            weight, posn = perform_likelihood_weighting(network, sorted_vars, queries, evidence, current_evidence)
            if weight < prev_weighted_likelihood:
                # Reject the next state because it has a lower probability than the previous one obtained from likelihood weighting
                current_evidence = old_evidence
            else:
                # Accept the next state
                prev_weighted_likelihood = weight
                prob_distribution[posn] += weight

    return prob_distribution / np.sum(prob_distribution)

In [2]:
import json

queries = [0]
evidence = {3:True,4:True}

with open('bn_test_1.json') as f:
    bayesian_network = json.load(f)
    print(estimate_metropolis_hastings(p=0.75, network=bayesian_network, queries=queries, evidence=evidence, iterations=10000))
    print(estimate_metropolis_hastings(p=0.85, network=bayesian_network, queries=queries, evidence=evidence, iterations=10000))
    print(estimate_metropolis_hastings(p=0.95, network=bayesian_network, queries=queries, evidence=evidence, iterations=10000))

[0.62421554 0.37578446]
[0.62947395 0.37052605]
[0.62979446 0.37020554]


In [22]:
import json

queries = [0, 3]
evidence = {2:True}

with open('bn_test_2.json') as f:
    bayesian_network = json.load(f)
    print(estimate_metropolis_hastings(p=0.75, network=bayesian_network, queries=queries, evidence=evidence, iterations=10000))
    print(estimate_metropolis_hastings(p=0.85, network=bayesian_network, queries=queries, evidence=evidence, iterations=10000))
    print(estimate_metropolis_hastings(p=0.95, network=bayesian_network, queries=queries, evidence=evidence, iterations=10000))

[0.34738775 0.21039387 0.13905487 0.30316352]
[0.35530454 0.21318999 0.13266003 0.29884544]
[0.36132509 0.20684929 0.13822199 0.29360364]


In [26]:
import json

queries = [1]
evidence = {2:False}

with open('bn_test_3.json') as f:
    bayesian_network = json.load(f)
    print(estimate_metropolis_hastings(p=0.75, network=bayesian_network, queries=queries, evidence=evidence, iterations=10000))
    print(estimate_metropolis_hastings(p=0.85, network=bayesian_network, queries=queries, evidence=evidence, iterations=10000))
    print(estimate_metropolis_hastings(p=0.95, network=bayesian_network, queries=queries, evidence=evidence, iterations=10000))

[0.67567946 0.32432054]
[0.66064759 0.33935241]
[0.64406969 0.35593031]


In [ ]:
import json

queries = [1, 3]
evidence = {2:False, 5:True}

with open('small_polytree.json') as f:
    bayesian_network = json.load(f)
    print(estimate_metropolis_hastings(p=0.75, network=bayesian_network, queries=queries, evidence=evidence, iterations=10000))
    print(estimate_metropolis_hastings(p=0.85, network=bayesian_network, queries=queries, evidence=evidence, iterations=10000))
    print(estimate_metropolis_hastings(p=0.95, network=bayesian_network, queries=queries, evidence=evidence, iterations=10000))

In [ ]:
import json

queries = [1, 3]
evidence = {2:False, 5:True}

with open('big_polytree.json') as f:
    bayesian_network = json.load(f)
    print(estimate_metropolis_hastings(p=0.75, network=bayesian_network, queries=queries, evidence=evidence, iterations=10000))
    print(estimate_metropolis_hastings(p=0.85, network=bayesian_network, queries=queries, evidence=evidence, iterations=10000))
    print(estimate_metropolis_hastings(p=0.95, network=bayesian_network, queries=queries, evidence=evidence, iterations=10000))